Unzip

In [ ]:
!unzip Localization_project-main.zip
%ls

Archive:  Localization_project-main.zip
   creating: Localization_project-main/
   creating: Localization_project-main/Ground/
   creating: Localization_project-main/Ground/all_gt/
  inflating: Localization_project-main/Ground/all_gt/1.csv  
  inflating: Localization_project-main/Ground/all_gt/10.csv  
  inflating: Localization_project-main/Ground/all_gt/11.csv  
  inflating: Localization_project-main/Ground/all_gt/12.csv  
  inflating: Localization_project-main/Ground/all_gt/13.csv  
  inflating: Localization_project-main/Ground/all_gt/14.csv  
  inflating: Localization_project-main/Ground/all_gt/15.csv  
  inflating: Localization_project-main/Ground/all_gt/16.csv  
  inflating: Localization_project-main/Ground/all_gt/17.csv  
  inflating: Localization_project-main/Ground/all_gt/18.csv  
  inflating: Localization_project-main/Ground/all_gt/19.csv  
  inflating: Localization_project-main/Ground/all_gt/2.csv  
  inflating: Localization_project-main/Ground/all_gt/20.csv  
  inflating: Lo

RSSI_Brute force

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Kernel, Hyperparameter, RBF, ConstantKernel, RationalQuadratic, Matern, DotProduct
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C,WhiteKernel , RationalQuadratic ,ExpSineSquared
from sklearn.preprocessing import StandardScaler

# Utility to split data into train and test sets
def testAndTrainData(df1, df2):
    tmp1 = df1['location'].to_numpy()
    y1 = df1['rssi_mean']
    y_train = y1.to_numpy()
    X1 = [list(map(int, i.split(","))) for i in tmp1]
    X_train = np.array(X1)

    tmp2 = df2['location'].to_numpy()
    y2 = df2['rssi_mean']
    y_test = y2.to_numpy()
    X2 = [list(map(int, i.split(","))) for i in tmp2]
    X_test = np.array(X2)

    return X_train, y_train, X_test, y_test

# def hybridGPKernal():
#     long_term_trend_kernel = 50.0 ** 2 * RBF(length_scale=50.0)
#     seasonal_kernel = (
#         2.0 ** 2
#         * RBF(length_scale=100.0)
#         * ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds="fixed")
#     )

#     irregularities_kernel = 0.5 ** 2 * RationalQuadratic(length_scale=1.0, alpha=1.0)

#     noise_kernel = 0.1 ** 2 * RBF(length_scale=0.1) + WhiteKernel(
#         noise_level=0.1 ** 2, noise_level_bounds=(1e-5, 1e5)
#     )

#     rssi_kernel = (
#         long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
#     )

#     return rssi_kernel

# Epanechnikov kernel for GPR
class EpanechnikovKernel(Kernel):
    def __init__(self, length_scale=1.0, length_scale_bounds=(1e-5, 1e5)):
        self.length_scale = length_scale
        self.length_scale_bounds = length_scale_bounds

    @property
    def hyperparameter_length_scale(self):
        return Hyperparameter("length_scale", "numeric", self.length_scale_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        X = np.atleast_2d(X)
        length_scale = self.length_scale

        if Y is None:
            Y = X
        else:
            Y = np.atleast_2d(Y)

        # Pairwise Euclidean distances
        dists_sq = np.sum(X**2, axis=1)[:, np.newaxis] + \
                   np.sum(Y**2, axis=1)[np.newaxis, :] - \
                   2 * np.dot(X, Y.T)
        dists = np.sqrt(np.maximum(dists_sq, 1e-12))
        dists_normalized = dists / length_scale

        # Epanechnikov kernel
        K = (3 / 4) * np.maximum(1 - dists_normalized**2, 0)

        if eval_gradient:
            mask = (dists_normalized <= 1)
            dK_dl = (3 / 4) * (2 * dists_normalized**2 / length_scale) * mask
            return K, dK_dl[..., np.newaxis]
        else:
            return K

    def diag(self, X):
        return np.full(X.shape[0], 0.75)

    def is_stationary(self):
        return True

    def __repr__(self):
        return f"EpanechnikovKernel(length_scale={self.length_scale})"

def calculate_distances(X):
    """Calculate distances from origin and center for enhanced features"""
    center = np.array([126.5, 121.5])
    distances = np.array([
        [euclidean(point, [0, 0]), euclidean(point, center)]
        for point in X
    ])
    return distances

# Main processing loop
floor = '/content/Localization_project-main/Second'
folder_path = floor + '/beacons_gt/'

test_location = [
    "72, 172","233, 107","171, 201","95, 50","82, 142","194, 228","115, 54","99, 24","17, 54","95, 40","72, 131","145, 189","189, 228","226, 217","171, 179","134, 54","238, 112","220, 157","184, 95","189, 242","65, 189","129, 57","80, 32","55, 32","7, 54","141, 54"
]

seasonal_kernel = (2.0 ** 2 * RBF(length_scale=100.0) * ExpSineSquared(length_scale=1.0, periodicity=1.0, periodicity_bounds="fixed"))
irregularities_kernel = 0.5 ** 2 * RationalQuadratic(length_scale=1.0, alpha=1.0)

# Kernel candidates to test
kernel_candidates = [
    # ConstantKernel() * Matern(nu=0.5) + WhiteKernel(),
    # ConstantKernel() * Matern(nu=1.5) + WhiteKernel(),
    # ConstantKernel() * Matern(nu=2.5) + WhiteKernel(),
    # ConstantKernel() * RBF() + WhiteKernel(),
    # ConstantKernel() * RationalQuadratic() + WhiteKernel(),
    # ConstantKernel() * EpanechnikovKernel() + WhiteKernel(),
    Matern(nu = 2.5) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    Matern(nu = 1.5) + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    DotProduct() + RBF(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    DotProduct() + EpanechnikovKernel(length_scale=1.0) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    DotProduct() + Matern(nu=1.5) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel,
    DotProduct() + Matern(nu=2.5) + ConstantKernel(1.0, (1e-3, 1e3)) * WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e5)) + seasonal_kernel + irregularities_kernel
    # hybridGPKernal(),
    # EpanechnikovKernel(length_scale=1.0) + WhiteKernel(noise_level=0.1),
    # Matern(nu = 2.5) + EpanechnikovKernel(length_scale=1.0) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e5)) + seasonal_kernel + irregularities_kernel,
    # Matern(nu = 1.5) + EpanechnikovKernel(length_scale=1.0) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e5)) + seasonal_kernel + irregularities_kernel
]

for filename in os.listdir(folder_path):
    print(f"Processing {filename}")
    if not filename.endswith('.csv'):
        continue

    # Load data
    df = pd.read_csv(os.path.join(folder_path, filename))
    v = filename.split('.csv')[0]

    # Split data
    df1 = df.loc[~df['location'].isin(test_location)]
    df2 = df.loc[df['location'].isin(test_location)].sort_values(by=['location'])
    X_train, y_train, X_test, y_test = testAndTrainData(df1, df2)

    # if len(df1) < 3:
    #     print(f"Skipping {v} - insufficient data ({len(df1)} train, {len(df2)} test)")
    #     continue

    # Feature engineering: Add distance-based features
    X_train_dist = calculate_distances(X_train)
    X_test_dist = calculate_distances(X_test)

    # Combine coordinates with distance features
    X_train_extended = np.hstack((X_train, X_train_dist))
    X_test_extended = np.hstack((X_test, X_test_dist))

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_extended)
    X_test_scaled = scaler.transform(X_test_extended)

    # Track best model for this beacon
    best_mae = float('inf')
    best_gp = None
    best_kernel = None

    # Try each kernel candidate
    for kernel in kernel_candidates:
        gp = GaussianProcessRegressor(
            kernel=kernel,
            n_restarts_optimizer=15,
            normalize_y=True,
            alpha=0.01  # Small noise for stability
        )

        # Fit and predict
        gp.fit(X_train_scaled, y_train)
        y_pred, _ = gp.predict(X_test_scaled, return_std=True)

        # Calculate MAE
        current_mae = mean_absolute_error(y_test, y_pred)

        # Track best kernel
        if current_mae < best_mae:
            best_mae = current_mae
            best_gp = gp
            best_kernel = kernel

    print(f"Best kernel for {v}: {best_kernel} with MAE: {best_mae:.2f}")

    # ================== DYNAMIC MESHGRID GENERATION ==================
    # Combine all data points to determine bounds
    all_points = np.vstack((X_train, X_test))

    # Get min/max coordinates with buffer
    min_x, min_y = np.min(all_points, axis=0)
    max_x, max_y = np.max(all_points, axis=0)

    # Add 10% buffer to the bounding box
    x_buffer = max(1, int(0.1 * (max_x - min_x)))
    y_buffer = max(1, int(0.1 * (max_y - min_y)))

    # Define bounds with buffer (clamped to 1-242/259 range)
    min_x = max(1, min_x - x_buffer)
    max_x = min(252, max_x + x_buffer)
    min_y = max(1, min_y - y_buffer)
    max_y = min(240, max_y + y_buffer)

    # Create dynamic meshgrid within bounds
    x1 = np.arange(min_x, max_x + 1).astype(int)
    x2 = np.arange(min_y, max_y + 1).astype(int)
    X0p, X1p = np.meshgrid(x1, x2)
    grid_points = np.c_[X0p.ravel(), X1p.ravel()]

    # Add distance features to grid points
    grid_distances = calculate_distances(grid_points)
    grid_points_extended = np.hstack((grid_points, grid_distances))
    grid_points_scaled = scaler.transform(grid_points_extended)

    # Predict using best model
    mean_pred, std_pred = best_gp.predict(grid_points_scaled, return_std=True)

    # Create full grid arrays with -99 default
    full_x = np.arange(1, 253)
    full_y = np.arange(1, 243)
    full_X0p, full_X1p = np.meshgrid(full_x, full_y)
    full_grid_points = np.c_[full_X0p.ravel(), full_X1p.ravel()]

    # Initialize full RSSI arrays with -99
    full_mean_pred = np.full(len(full_grid_points), -102.0)
    full_std_pred = np.full(len(full_grid_points), -102.0)

    # Create mapping from coordinate to index
    coord_to_index = {(x, y): i for i, (x, y) in enumerate(full_grid_points)}

    # Map predictions to full grid
    for (x, y), mean_val, std_val in zip(grid_points, mean_pred, std_pred):
        idx = coord_to_index.get((x, y), None)
        if idx is not None:
            full_mean_pred[idx] = mean_val
            full_std_pred[idx] = std_val

    # Prepare data for saving
    data = [
        [f"{x}, {y}", round(mean, 2), round(std, 2)]
        for (x, y), mean, std in zip(full_grid_points, full_mean_pred, full_std_pred)
    ]
    # ================== END DYNAMIC MESHGRID ==================

    df_out = pd.DataFrame(data, columns=["location", "rssi_mean", "rssi_std"])
    df_out.to_csv(f'/content/Localization_project-main/Second/beacons_pd/{v}.csv', index=False)

    # Final evaluation
    y_pred, _ = best_gp.predict(X_test_scaled, return_std=True)
    final_mae = mean_absolute_error(y_test, y_pred)
    print(f"  Final MAE: {final_mae:.2f}")
    print(f"  Grid size: Original (252x242) | Dynamic: ({min_x}-{max_x})x({min_y}-{max_y})")

Processing IW25030922_rssi.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/

Best kernel for IW25030922_rssi: Matern(length_scale=1, nu=1.5) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 1.41
  Final MAE: 1.41
  Grid size: Original (252x242) | Dynamic: (170-181)x(133-205)
Processing IW25030905_rssi.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kerne

Best kernel for IW25030905_rssi: Matern(length_scale=1, nu=1.5) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 4.04
  Final MAE: 4.04
  Grid size: Original (252x242) | Dynamic: (17-102)x(20-57)
Processing IW25030919_rssi.csv


...

Best kernel for IW25030973_rssi: Matern(length_scale=1, nu=2.5) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 0.22
  Final MAE: 0.22
  Grid size: Original (252x242) | Dynamic: (64-81)x(22-33)
Processing IW25030903_rssi.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gauss

Best kernel for IW25030903_rssi: Matern(length_scale=1, nu=2.5) + EpanechnikovKernel(length_scale=1.0) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 14.23
  Final MAE: 14.23
  Grid size: Original (252x242) | Dynamic: (7-131)x(53-55)
Processing IW25030913_rssi.csv


/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sk

Best kernel for IW25030913_rssi: DotProduct(sigma_0=1) + Matern(length_scale=1, nu=1.5) + 1**2 * WhiteKernel(noise_level=0.1) + 2**2 * RBF(length_scale=100) * ExpSineSquared(length_scale=1, periodicity=1) + 0.5**2 * RationalQuadratic(alpha=1, length_scale=1) with MAE: 7.46
  Final MAE: 7.46
  Grid size: Original (252x242) | Dynamic: (53-84)x(125-199)


Localization_2

In [ ]:
#!/usr/bin/env python
import os
from IPython.display import display
from itertools import product
import numpy as np
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C,WhiteKernel , RationalQuadratic ,ExpSineSquared
# %matplotlib notebook


# Directory containing beacon files
def GroundTruthData(directory_gt, floor,test_location,num_files):

    # Initialize an empty list to hold dataframes
    dataframes_gt = []
    # Loop through each beacon file
    for filename in os.listdir(directory_gt):
        # print(filename)
        file_path = os.path.join(directory_gt, filename)
        df = pd.read_csv(file_path)
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        #df = df.drop(['rssi_std'], axis=1) # Drop 'rssi_std' column
        dataframes_gt.append(df)  # Append dataframe to the list

    dataframe_gt = pd.concat([df.set_index('location') for df in dataframes_gt], axis=1, join='outer').reset_index()
    dataframe_gt.fillna('-100', inplace=True)
    # print(dataframe_gt.round(0))
    dataframe_gt.to_csv(floor+'/all_scan_sample.csv', index=False)

    ### Randomly sampled 6 test location out of total 29 location and 3 outlier = 32 points####
    test_data = dataframe_gt.loc[dataframe_gt['location'].isin(test_location)]
    test_data = test_data.sort_values(by=['location'])
    temp = test_data.location.str.split(",", n = -1, expand = True)
    # print(dataframe_gt)
    test_data['cordX'] = temp[0].astype(int)
    test_data['cordY'] = temp[1].astype(int)
    # print(data.head())
    X_gt= test_data.iloc[:, 1:num_files]
    y1_gt = test_data['cordX']
    y2_gt = test_data['cordY']
    test_data.head()
    test_data.shape

    return X_gt, y1_gt, y2_gt


def Custom_PD_data(directory_pd, floor,test_location,num_files):

    # Initialize an empty list to hold dataframes
    dataframes_pd = []
    # Loop through each beacon file
    for filename in os.listdir(directory_pd):
        # print(filename)
        file_path = os.path.join(directory_pd, filename)
        df = pd.read_csv(file_path)
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        df = df.drop(['rssi_std'], axis=1) # Drop 'rssi_std' column
        dataframes_pd.append(df)  # Append dataframe to the list

    dataframe_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()

    ### Randomly sampled 6 test location out of total 29 location and 3 outlier = 32 points####
    test_data = dataframe_pd.loc[dataframe_pd['location'].isin(test_location)]
    test_data = test_data.sort_values(by=['location'])
    temp = test_data.location.str.split(",", n = -1, expand = True)
    test_data['cordX'] = temp[0].astype(int)
    test_data['cordY'] = temp[1].astype(int)
    # print(data.head())
    X_pd_custom= test_data.iloc[:, 1:num_files]
    y1_pd_custom = test_data['cordX']
    y2_pd_custom = test_data['cordY']
    test_data.head()
    test_data.shape

    return X_pd_custom, y1_pd_custom, y2_pd_custom

### beacons_pd folder include predicted beacons rssi data ########
def GPPredictedData(directory_pd, num_files):
    dataframes_pd = []
    for filename in os.listdir(directory_pd):
        file_path = os.path.join(directory_pd, filename)
        # Read CSV file
        df = pd.read_csv(file_path)
        # Rename column
        df.rename(columns={'rssi_mean':  filename.split('.csv')[0]}, inplace=True)
        # Drop 'rssi_std' column
        df = df.drop(['rssi_std'], axis=1)
        # Append dataframe to the list
        dataframes_pd.append(df)
    # Merge all dataframes on 'location'
    dataframes_pd = pd.concat([df.set_index('location') for df in dataframes_pd], axis=1, join='inner').reset_index()
    new = dataframes_pd.location.str.split(",", n = -1, expand = True)
    dataframes_pd['cordX'] = new[0].astype(int)
    dataframes_pd['cordY'] = new[1].astype(int)
    X_pd = dataframes_pd.iloc[:, 1:num_files]
    y1_pd = dataframes_pd['cordX']
    y2_pd = dataframes_pd['cordY']

    dataframes_pd.head()
    dataframes_pd.shape

    return X_pd, y1_pd, y2_pd

#################### KNN training for X-axis ####################
def knnXaxis(X_train, y1_train, X_test,y1_test, y1_gt):
    print("---------KNN training for X-axis result---------" )
    # print(X_test, y1_test)
    knn_classifier = KNeighborsClassifier(n_neighbors = 3)
    # print(X_train, y1_train)
    knn_classifier.fit(X_train, y1_train.values.ravel())
    # test using GP predicted test dataset
    y1_pred=knn_classifier.predict(X_test)
    y1_pred_custom=knn_classifier.predict(X_pd_custom)
    # print(y1_pred_custom)
    # print(y1_pd_custom)
    # knn_accuracy=accuracy_score(y1_test, y1_pred)
    # print('\nThe Models Accuracy is', knn_accuracy)
    MAE1 = mean_absolute_error(y1_pred,y1_test)
    MAE2 = mean_absolute_error(y1_pred_custom,y1_pd_custom)
    print("Predicted data knn-mae-X-axis :\t",MAE1*0.305, "m")
    print("Custom Predicted data knn-mae-X-axis (0 to 15 ft):\t",MAE2*0.305, "m")
    # test using ground truth test dataset
    # print("Inputs==========",X_gt)
    y1_pred_gt=knn_classifier.predict(X_gt)
    # print("Outputs======",y1_pred_gt)
    # # knn_accuracy=accuracy_score(y1_gt, y1_pred_gt)
    # # print('\nThe Models Accuracy for ground truth values, is', knn_accuracy)
    # print(y1_gt,y1_pred_gt)
    MAE3 = mean_absolute_error(y1_pred_gt,y1_gt)
    print("Ground truth data knn-mae-X-axis :\t",MAE3*0.305, "m")

    return MAE1, MAE2, MAE3, y1_pred, y1_pred_custom, y1_pred_gt


#################### KNN training for Y-axis ####################
def knnYaxis(X_train, y2_train, X_test, y2_test,y2_gt):
    print("---------KNN training for Y-axis result---------" )
    knn_classifier = KNeighborsClassifier(n_neighbors = 3)
    knn_classifier.fit(X_train, y2_train.values.ravel())
    # test using GP predicted test dataset
    y2_pred=knn_classifier.predict(X_test)
    y2_pred_custom=knn_classifier.predict(X_pd_custom)
    #print(y2_pred)
    # knn_accuracy=accuracy_score(y2_test, y2_pred)
    # print('\nThe Models Accuracy is', knn_accuracy)

    MAE4 = mean_absolute_error(y2_pred,y2_test)
    MAE5 = mean_absolute_error(y2_pred_custom,y2_pd_custom)
    print("Predicted data knn-mae-Y-axis :\t",MAE4*0.305, "m")
    print("Custom Predicted data knn-mae-Y-axis (0 to 15 ft):\t",MAE5*0.305, "m")
    # test using ground truth test dataset
    y2_pred_gt=knn_classifier.predict(X_gt)
    MAE6 = mean_absolute_error(y2_pred_gt,y2_gt)
    # print(y2_gt,y2_pred_gt)
    print("Ground truth data knn-mae-Y-axis :\t",MAE6*0.305, "m")

    return MAE4, MAE5, MAE6, y2_pred, y2_pred_custom, y2_pred_gt


########## KNN - Localization performance #########################
########### Part 1: Result on GP predicted test dataset ##################
def knnLocalizationPredictedData(y1_test, y2_test, X_test, y1_pred, y2_pred, path_3):
    r1 = pd.concat([y1_test, y2_test, X_test], axis=1)
    # r1.rename(columns={'cordX': 'cordXold', 1: 'cordYold'}, inplace=True)
    # r1.drop(columns=["cordX", "cordY"], inplace=True)
    r2 = pd.concat([pd.DataFrame({'SP_cordX':y1_pred}),pd.DataFrame({'SP_cordY':y2_pred})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #display(result)
    # euclidean distance error in a grid world
    # result = result.rename(columns={0: 'cordXold', 1: 'cordYold'}, inplace=True)
    result['dist_localization_error_pd']  = np.round(np.sqrt((result['cordX'] - result['SP_cordX'])**2 + (result['cordY'] - result['SP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['SP_cordX'].map(str)).str.cat(result['SP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_pd']]
    # print("Part 1: Result on GP predicted test dataset ==========")

    print("Localziation_mse_knn_pd_predicted", np.mean(result['dist_localization_error_pd'])*0.305, "m")
    # print("Localization error WRT Predicted data", result['dist_localization_error_pd']*0.305, "m")
    #result.to_csv(path_3+'/knn_result_pd_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_pd'])


def knnLocalizationcustomPD(y1_pd_custom, y2_pd_custom, X_pd_custom, y1_pred_custom, y2_pred_custom):
    r1 = pd.concat([y1_pd_custom, y2_pd_custom, X_pd_custom], axis=1)
    # r1.rename(columns={'cordX': 'cordXold', 1: 'cordYold'}, inplace=True)
    # r1.drop(columns=["cordX", "cordY"], inplace=True)
    r2 = pd.concat([pd.DataFrame({'SP_cordX':y1_pred_custom}),pd.DataFrame({'SP_cordY':y2_pred_custom})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #display(result)
    # euclidean distance error in a grid world
    # result = result.rename(columns={0: 'cordXold', 1: 'cordYold'}, inplace=True)
    result['dist_localization_error_pd']  = np.round(np.sqrt((result['cordX'] - result['SP_cordX'])**2 + (result['cordY'] - result['SP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['SP_cordX'].map(str)).str.cat(result['SP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_pd']]
    # print("Part 1: Result on GP predicted test dataset ==========")

    print("Localziation_mse_knn_custom_pd_predicted (0 to 15 ft)", np.mean(result['dist_localization_error_pd'])*0.305, "m")
    # print("Localization error WRT Predicted data", result['dist_localization_error_pd']*0.305, "m")
    #result.to_csv(path_3+'/knn_result_pd_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_pd'])



########## Part 2: Result on Ground truth test dataset ###################
def knnLocalizationGroundTruthData(y1_gt, y2_gt, X_gt, y1_pred_gt, y2_pred_gt, path_4):
    r1 = pd.concat([y1_gt, y2_gt, X_gt], axis=1)
    r2 = pd.concat([pd.DataFrame({'GPP_cordX':y1_pred_gt}),pd.DataFrame({'GPP_cordY':y2_pred_gt})], axis=1)
    result = pd.concat([r1,r2.set_index(r1.index)], axis=1)
    #print(result)
    # euclidean distance error in a grid world
    result['dist_localization_error_gt']  = np.round(np.sqrt((result['cordX'] - result['GPP_cordX'])**2 + (result['cordY'] - result['GPP_cordY'])**2),2)
    result['loc'] = (result['cordX'].map(str)).str.cat(result['cordY'].map(str),sep=",")
    result['Pred_loc'] = (result['GPP_cordX'].map(str)).str.cat(result['GPP_cordY'].map(str),sep=",")
    result = result[['loc','Pred_loc','dist_localization_error_gt']]
    # print("Part 2: Result on Ground truth test dataset ==========")

    print("Localziation_mse_knn_gt_predicted", np.mean(result['dist_localization_error_gt'])*0.305, "m")
    # print("Localization error WRT Ground truth data", result['dist_localization_error_gt']*0.305, "m")
    #result.to_csv(path_4+'/knn_result_gt_predicted.csv', index=False)

    return np.mean(result['dist_localization_error_gt'])


floor = '/content/Localization_project-main/Second'
path_1 ='/content/Localization_project-main/Second/beacons_gt'

test_location_pd = [
    "174, 143", "54, 44", "99, 25", "163, 74", "241, 132", "76, 60", "98, 26", "167, 221", "200, 161", "151, 193", "191, 128", "191, 231", "27, 40", "131, 57", "187, 235", "58, 35", "102, 191", "122, 44", "53, 54", "185, 93", "43, 56", "198, 93", "180, 104", "172, 209", "195, 161", "72, 174", "172, 139", "121, 25", "32, 51", "27, 33", "237, 100", "88, 123", "86, 139", "103, 56", "100, 190", "115, 33", "107, 53", "128, 188", "44, 52", "65, 144", "21, 56", "195, 232", "150, 189", "205, 221", "99, 185", "72, 31", "148, 192", "76, 120", "127, 188", "168, 213", "247, 123", "112, 54", "20, 34", "183, 136", "132, 190", "241, 119", "169, 183", "75, 186", "73, 150", "200, 214", "226, 217", "181, 92", "70, 94", "157, 187", "98, 188", "137, 187", "31, 30", "71, 31", "243, 125", "152, 186", "183, 106", "72, 184", "83, 127", "67, 187", "193, 104", "74, 57", "119, 191", "206, 225", "169, 145", "71, 119", "204, 109", "179, 85", "174, 78", "120, 44", "43, 58", "111, 22", "114, 23", "20, 39", "74, 111", "126, 188", "167, 202", "96, 51", "243, 97", "71, 58", "62, 29", "249, 108", "186, 124", "69, 55", "76, 61", "74, 67",
    "81, 37", "157, 46", "197, 101", "92, 137", "41, 198", "81, 85", "189, 222", "50, 184", "94, 16", "40, 194", "245, 141", "175, 221", "57, 27", "35, 40", "67, 80", "73, 26", "29, 63", "132, 48", "1, 49", "63, 40", "162, 137", "81, 178", "61, 183", "163, 214", "141, 181", "89, 41", "202, 161", "209, 229", "109, 11", "180, 80", "228, 105", "172, 154", "181, 78", "59, 143", "186, 140", "176, 157", "69, 48", "185, 250", "93, 15", "178, 171", "109, 10", "64, 46", "206, 110", "258, 120", "120, 62", "184, 249", "51, 195", "101, 35", "220, 163", "80, 158", "170, 87", "200, 151", "163, 205", "81, 105", "165, 177", "113, 59", "77, 68", "183, 83", "199, 83", "177, 218", "208, 88", "79, 116", "165, 147", "96, 197", "187, 90", "72, 26", "77, 18", "133, 63", "66, 99", "230, 209", "119, 180", "12, 62", "86, 198", "217, 208", "3, 59", "55, 198", "122, 63", "39, 182", "178, 243", "24, 28", "42, 63", "190, 212", "97, 136", "211, 110", "239, 145", "78, 96", "154, 46", "239, 120", "198, 164", "53, 59", "49, 62", "69, 139", "164, 230", "179, 122", "149, 195", "18, 30", "49, 26", "181, 189", "144, 47", "68, 85",
    "183, 194", "183, 161", "61, 83", "59, 162", "163, 53", "161, 147", "58, 144", "231, 226", "237, 213", "84, 174", "104, 11", "103, 200", "160, 171", "16, 30", "181, 150", "130, 40", "206, 165", "239, 87", "184, 218", "163, 89", "83, 81", "229, 151", "159, 230", "7, 67", "157, 201", "13, 41", "236, 225", "83, 101", "55, 65", "152, 201", "45, 66", "162, 174", "146, 45", "160, 206", "190, 82", "159, 214", "41, 65", "88, 64", "251, 93", "204, 167", "129, 200", "81, 202", "182, 202", "120, 65", "188, 158", "208, 151", "232, 94", "36, 197", "33, 184", "83, 107", "89, 64", "245, 89", "80, 43", "200, 132", "180, 249", "61, 131", "212, 167", "224, 115", "85, 111", "61, 92", "204, 100", "229, 206", "89, 201", "72, 14", "242, 147", "106, 39", "57, 65", "167, 127", "190, 138", "159, 216", "117, 201", "42, 23", "154, 43", "156, 178", "60, 129", "72, 204", "92, 155", "232, 90", "35, 184", "188, 159", "83, 200", "165, 251", "96, 112", "61, 70", "169, 107", "199, 123", "63, 21", "110, 8", "60, 178", "101, 127", "123, 66", "218, 229", "203, 121", "224, 168", "82, 73", "152, 179", "82, 76", "176, 72", "57, 149", "169, 62"
]
test_location_ground = ["99, 24", "107, 24", "118, 31", "121, 54", "95, 54", "72, 54", "65, 54", "55, 54", "47, 54", "17, 54", "36, 54", "95, 40", "118, 24", "72, 81", "112, 19", "72, 102", "129, 57", "72, 123", "72, 131", "151, 54", "72, 142", "134, 54", "153, 64", "72, 162", "72, 184", "163, 74", "45, 189", "175, 86", "82, 189", "184, 95", "192, 103", "153, 189", "171, 189", "200, 111", "171, 201", "188, 127", "171, 209", "180, 127", "171, 139", "180, 139", "135, 189", "198, 157", "126, 189", "180, 103", "195, 92", "214, 101", "214, 92", "233, 107", "241, 99", "238, 112", "246, 112", "252, 112", "171, 225", "245, 122", "171, 234", "241, 128", "194, 228", "241, 137", "189, 235", "220, 157", "189, 242", "80, 23", "204, 228", "204, 217", "80, 32", "71, 32", "55, 32", "48, 32", "30, 32", "24, 37", "94, 24", "112, 24", "118, 34", "118, 44", "115, 54", "105, 54", "95, 50", "92, 54", "82, 54", "46, 54", "31, 54", "27, 54", "7, 54", "55, 42", "77, 32", "65, 32", "72, 61", "72, 71", "72, 91", "72, 101", "72, 113", "72, 132", "76, 142", "82, 142", "82, 150", "72, 152", "68, 145", "82, 131", "90, 131", "90, 122", "72, 172", "72, 182", "171, 179", "55, 189", "171, 169", "65, 189", "171, 159", "171, 149", "72, 189", "171, 215", "163, 189", "72, 193", "145, 189", "86, 189", "116, 189", "96, 189", "106, 189", "171, 199", "204, 227", "189, 228", "217, 217", "181, 228", "226, 217", "169, 225", "171, 240", "194, 221", "141, 54", "129, 54", "160, 71", "182, 93", "170, 81"
]

num_files = len(os.listdir(path_1))+1    # Count the number of files
test_location = test_location_ground


#print(test_location)
X_gt, y1_gt, y2_gt = GroundTruthData(path_1, floor, test_location,num_files)

path_2 = '/content/Localization_project-main/Second/beacons_pd'
num_files = len(os.listdir(path_2))+1    # Count the number of files
X_pd_custom, y1_pd_custom, y2_pd_custom = Custom_PD_data(path_2,floor,test_location_pd,num_files)
X_pd, y1_pd, y2_pd = GPPredictedData(path_2,num_files)
#print(num_files)
#print(X_pd_custom)
# #### Split the data set for training and testing ###########
X_train, X_test, y1_train, y1_test = train_test_split(X_pd, y1_pd,test_size=0.2, random_state= 10)
mae_pd_x, mae_pd_custom_x, mae_gt_x, y1_pred, y1_pred_custom, y1_pred_gt = knnXaxis(X_train, y1_train, X_test,y1_test, y1_gt)
#print(y1_test)
# print(X_pd)
# #### Split the data set for training and testing ###########
X_train, X_test, y2_train, y2_test = train_test_split(X_pd, y2_pd,test_size=0.2, random_state= 10)
mae_pd_y, mae_pd_custom_y, mae_gt_y, y2_pred, y2_pred_custom, y2_pred_gt =  knnYaxis(X_train, y2_train, X_test, y2_test,y2_gt)
#print(y1_test,y2_test)
# print(y1_gt.to_frame()['cordX'], y2_gt.to_frame()['cordY'])
#y1_y2_gt = pd.concat([y1_gt.to_frame(), y2_gt.to_frame()], axis=1)
# print(y1_y2_gt)
y1_y2_pd_gt = pd.concat([pd.DataFrame(y1_pred_gt), pd.DataFrame(y2_pred_gt)], axis=1)
y1_y2_pd_gt.rename(columns={'0': 'cordX', '0': 'cordY'})
# print(pd.DataFrame(y1_pred_gt))
# print(y1_y2_pd_gt)
# print(pd.concat([y1_y2_gt, y1_y2_pd_gt], axis=1))
print("---------KNN Localization performance evaluatiion ---------" )
path_3 = floor+'/results'
path_4 = floor+'/results'
mse_knn_pd = knnLocalizationPredictedData(y1_test, y2_test, X_test, y1_pred, y2_pred, path_3)
mse_knn_custom_pd = knnLocalizationcustomPD(y1_pd_custom, y2_pd_custom, X_pd_custom, y1_pred_custom, y2_pred_custom)
mse_knn_gt = knnLocalizationGroundTruthData(y1_gt, y2_gt, X_gt, y1_pred_gt, y2_pred_gt, path_4)

/tmp/ipython-input-11-453921332.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-100' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe_gt.fillna('-100', inplace=True)


---------KNN training for X-axis result---------
Predicted data knn-mae-X-axis :	 7.934226039189965 m
Custom Predicted data knn-mae-X-axis (0 to 15 ft):	 1.4181462585034013 m
Ground truth data knn-mae-X-axis :	 1.7590697674418603 m
---------KNN training for Y-axis result---------
Predicted data knn-mae-Y-axis :	 9.46280191850455 m
Custom Predicted data knn-mae-Y-axis (0 to 15 ft):	 2.424438775510204 m
Ground truth data knn-mae-Y-axis :	 1.503720930232558 m
---------KNN Localization performance evaluatiion ---------
Localziation_mse_knn_pd_predicted 13.786415602197263 m
Localziation_mse_knn_custom_pd_predicted (0 to 15 ft) 3.466916326530613 m
Localziation_mse_knn_gt_predicted 2.701874418604651 m
